### CREATE DAILY INPUT FOR THE CNN MODEL

In [5]:

import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plot
import rioxarray as xr
import datetime
import psycopg2
from geopandas import GeoSeries

import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import psycopg2
import shapely.wkb as wkb
from geopandas import GeoSeries

In [9]:
#retrieve the list of stations in the metadata database.

conn = psycopg2.connect(host="10.8.244.31",
                   database="climate_data",
                   user="ado_user",
                   password="hydro#ado",
                   port=5432)
cur = conn.cursor()

# get the metadata
query = f"""
        SELECT "id_station" FROM "hydrology"."metadata_disc"
        """
df = pd.read_sql_query(query, conn)

# close the connection when finished
cur.close()
conn.close()
LIST = list(df.id_station)
tif_fileName =  'Z:\ADO\EU_DEM\eudem_WGS84_adoext.tif'
dem=xr.open_rasterio(tif_fileName)

In [10]:
df

,id_station
0,ADO_DSC_ITH5_0001
1,ADO_DSC_ITH5_0002
2,ADO_DSC_ITH5_0003
3,ADO_DSC_ITH5_0004
4,ADO_DSC_ITH5_0005
5,ADO_DSC_ITH5_0006
6,ADO_DSC_ITH5_0007
7,ADO_DSC_ITH5_0008


In [11]:
table=pd.DataFrame(data=None)
    # Open the shape file

conn = psycopg2.connect(host="10.8.244.31",
                   database="climate_data",
                   user="ado_user",
                   password="hydro#ado",
                   port=5432)
cur = conn.cursor()
    
    
for STAT_CODE in LIST:
    
    # get the metadata
    query = f"""
            SELECT "geom" FROM "hydrology"."metadata_disc" WHERE "id_station" = '{STAT_CODE}'    
            """
    df = pd.read_sql_query(query, conn)
    
    shp=GeoSeries(wkb.loads(df.geom[0], hex=True))
    shp=shp.set_crs("EPSG:4326")
    
    h_station=dem.sel(x=shp[0].x,y=shp[0].y,method='nearest').values[0]

    update = {}
    update['name'] = STAT_CODE
    update['h']=h_station
    
    table = table.append(update,ignore_index=True)
    

In [12]:
table

,name,h
0,ADO_DSC_ITH5_0001,21.585526
1,ADO_DSC_ITH5_0002,14.239743
2,ADO_DSC_ITH5_0003,29.443939
3,ADO_DSC_ITH5_0004,5.580721
4,ADO_DSC_ITH5_0005,42.813999
5,ADO_DSC_ITH5_0006,3.534000
6,ADO_DSC_ITH5_0007,6.214500
7,ADO_DSC_ITH5_0008,48.115002


In [13]:
table.to_csv('STATIONS_HEIGHT_emilia.csv')